<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/MultiOutputRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
import numpy as np
import warnings;warnings.simplefilter('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression,Lasso,Ridge
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from math import sqrt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# some function

In [0]:
def mape_vectorized_v2(a, b):
    a = np.array(a)
    b = np.array(b) 
    mask = a != 0
    return (np.fabs(a - b)/a)[mask].mean()

In [0]:
def metric(y_t,y_p,name):
  res = {'R2':r2_score(y_t,y_p),
         'RMSE':sqrt(mean_squared_error(y_t,y_p)),
         'MAPE':mape_vectorized_v2(y_t,y_p)
         }
  return pd.DataFrame(res,index=[name])

In [0]:
def modeling(my_model,train,test):
  model = MultiOutputRegressor(my_model)
  model.fit(train[features],train[y_columns])
  y_pred = pd.DataFrame(model.predict(test[features]),columns=y_columns)
  res = pd.DataFrame()
  for y_name in y_columns:
    res = res.append(metric(test[y_name],y_pred[y_name],y_name))
  return res

# load data

In [5]:
data = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/test_250.csv',index_col=0)
train = data.loc[data.index %5 != 0]
test = data.loc[data.index %5 == 0].drop(95,axis=0)
print(train.shape)
print(test.shape)

(200, 27)
(49, 27)


# features , y_columns

In [0]:
cols = train.columns.tolist()
features = cols[:4]
y_columns = cols[4:]

# scaler

In [0]:
scaler_x = StandardScaler()
train[features] = scaler_x.fit_transform(train[features])
test[features] = scaler_x.transform(test[features])

# modeling

In [8]:
SVR_res = modeling(SVR(),train,test)
SVR_res.columns = ['SVR_'+i for i in SVR_res.columns.tolist()]

cat_res = modeling(CatBoostRegressor(learning_rate = 0.1,n_estimators=100,verbose=0),train,test)
cat_res.columns = ['CAT_'+i for i in cat_res.columns.tolist()]

xgb_res = modeling(XGBRegressor(learning_rate=0.1,n_estimators=100,objective='reg:squarederror'),train,test)
xgb_res.columns = ['XGB_'+i for i in xgb_res.columns.tolist()]

lr_res = modeling(LinearRegression(),train,test)
lr_res.columns = ['LR_'+i for i in lr_res.columns.tolist()]

all_res = SVR_res.join(cat_res).join(xgb_res).join(lr_res)
all_res = all_res[['SVR_R2','CAT_R2','XGB_R2','LR_R2']]
all_res['best_R2(max)'] = all_res.max(axis=1)
all_res.loc['AVG'] = all_res.mean(axis=0)
all_res.style.background_gradient(axis=1)

,SVR_R2,CAT_R2,XGB_R2,LR_R2,best_R2(max)
C5NP,0.964834,0.97502,0.970972,0.922175,0.97502
C5IP,0.949602,0.974611,0.961257,0.878052,0.974611
C5N,0.941323,0.949621,0.939434,0.926945,0.949621
C6NP,0.878527,0.862887,0.858899,0.787674,0.878527
C6IP,0.912735,0.910867,0.892215,0.894168,0.912735
C6N,0.674647,0.783154,0.860136,0.581286,0.860136
C6A,0.781986,0.729212,0.261147,0.447705,0.781986
C7NP,0.752797,0.758624,0.669579,0.399624,0.758624
C7IP,0.531363,0.666713,0.661991,0.16588,0.666713
C7N,0.849007,0.934483,0.925443,0.806473,0.934483


In [9]:
SVR_res = modeling(SVR(),train,test)
SVR_res.columns = ['SVR_'+i for i in SVR_res.columns.tolist()]

cat_res = modeling(CatBoostRegressor(learning_rate = 0.1,n_estimators=100,verbose=0),train,test)
cat_res.columns = ['CAT_'+i for i in cat_res.columns.tolist()]

xgb_res = modeling(XGBRegressor(learning_rate=0.1,n_estimators=100,objective='reg:squarederror'),train,test)
xgb_res.columns = ['XGB_'+i for i in xgb_res.columns.tolist()]

lr_res = modeling(LinearRegression(),train,test)
lr_res.columns = ['LR_'+i for i in lr_res.columns.tolist()]

all_res = SVR_res.join(cat_res).join(xgb_res).join(lr_res)
all_res = all_res[['SVR_RMSE','CAT_RMSE','XGB_RMSE','LR_RMSE']]
all_res['best_RMSE(min)'] = all_res.min(axis=1)
all_res.loc['AVG'] = all_res.mean(axis=0)
all_res.style.background_gradient(axis=1)

,SVR_RMSE,CAT_RMSE,XGB_RMSE,LR_RMSE,best_RMSE(min)
C5NP,0.737232,0.621353,0.669817,1.09674,0.621353
C5IP,0.671719,0.476763,0.588951,1.04489,0.476763
C5N,0.0764661,0.0708534,0.0776873,0.0853216,0.0708534
C6NP,0.682055,0.724632,0.735095,0.901739,0.682055
C6IP,0.781568,0.789889,0.868615,0.860707,0.781568
C6N,1.03902,0.848247,0.681239,1.1787,0.681239
C6A,0.416794,0.464509,0.767288,0.663384,0.416794
C7NP,0.419124,0.414156,0.484563,0.653172,0.414156
C7IP,1.07725,0.908459,0.914873,1.43718,0.908459
C7N,2.27268,1.49705,1.597,2.57295,1.49705


In [10]:
SVR_res = modeling(SVR(),train,test)
SVR_res.columns = ['SVR_'+i for i in SVR_res.columns.tolist()]

cat_res = modeling(CatBoostRegressor(learning_rate = 0.1,n_estimators=100,verbose=0),train,test)
cat_res.columns = ['CAT_'+i for i in cat_res.columns.tolist()]

xgb_res = modeling(XGBRegressor(learning_rate=0.1,n_estimators=100,objective='reg:squarederror'),train,test)
xgb_res.columns = ['XGB_'+i for i in xgb_res.columns.tolist()]

lr_res = modeling(LinearRegression(),train,test)
lr_res.columns = ['LR_'+i for i in lr_res.columns.tolist()]

all_res = SVR_res.join(cat_res).join(xgb_res).join(lr_res)
all_res = all_res[['SVR_MAPE','CAT_MAPE','XGB_MAPE','LR_MAPE']]
all_res['best_MAPE(min)'] = all_res.min(axis=1)
all_res.loc['AVG'] = all_res.mean(axis=0)
all_res.style.background_gradient(axis=1)

,SVR_MAPE,CAT_MAPE,XGB_MAPE,LR_MAPE,best_MAPE(min)
C5NP,0.840257,0.728842,0.716915,0.905873,0.716915
C5IP,1.10685,1.1013,0.925822,1.52009,0.925822
C5N,1.5949,0.837499,0.648161,1.98395,0.648161
C6NP,0.47897,0.334984,0.204009,1.60398,0.204009
C6IP,0.601792,0.518139,0.31477,2.35637,0.31477
C6N,0.140335,0.0967605,0.0832528,0.164453,0.0832528
C6A,0.455041,0.51885,0.633327,1.38005,0.455041
C7NP,0.0578097,0.0524001,0.0622863,0.0941402,0.0524001
C7IP,0.117675,0.0952121,0.0988952,0.215602,0.0952121
C7N,0.0963711,0.0656128,0.0756156,0.150478,0.0656128
